# Form Recognizerでレシートを分析する

<p style='text-align:center'><img src='./images/receipt_analysis.jpg' alt='レシートを持つロボット'/></p>

コンピュータビジョンの人工知能（AI）分野では、印刷された文書や手書きの文書を読み取るために光学式文字認識（OCR）が一般的に使用されています。多くの場合、テキストは単に文書から、さらなる処理や分析に使用できる形式に抽出されます。

より高度な OCR のシナリオは、発注書や請求書などのフォームから、フォームのフィールドが何を表しているかを意味的に理解した上で情報を抽出することです。**Form Recognizer**サービスは、この種のAI問題のために特別に設計されています。

## レシートを見る

この例では、レシートを分析するために Form Recognizer のビルトインモデルを使用します。

下のセルの左上にある緑色の <span style="color:green">&#9655;</span> ボタンをクリックして実行し、Form Recognizer を使ってレシートを分析する例を見てみましょう。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# レシート画像を読み込んで表示する
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## Form Recognizer リソースを作成する

Azureサブスクリプションでフォームレコグナイザーリソースを作成することから始めます。

1. 別のブラウザタブで、https://portal.azure.com の Azure ポータルを開き、Microsoft アカウントでサインインします。
2. **+リソースの作成**を選択し、*Form Recognizer*を検索します。
3. サービスのリストで、***Form Recognizer**を選択します。
4. **Form Recognizer**ブレードで、**Create**を選択します。
5. **Create**ブレードで、以下の詳細を入力し、**Create**を選択します。
  * **Name**: サービスのユニークな名前
  * **Subscription**: あなたのAzureサブスクリプション
  * **Location**: 利用可能な場所
  * **Pricing tier**: F0
  * **Resource Group**: 以前に使用した既存のリソースグループ
  * **I confirm I have read and understood the notice below**: 選択.
6. サービスが作成されるのを待ちます。
7. Azureポータルで新しく作成したフォームレコグナイザーサービスを表示し、**クイックスタート**ページで**Key1**と**Endpoint**の値をコピーして、**YOUR_FORM_KEY**と**YOUR_FORM_ENDPOINT**を置き換えて、以下のコードセルに貼り付けます。

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

これでCognitive Servicesのセットアップが完了しました。

そしてPythonから本演習を実行するには、まずAzureの関連パッケージをインストールする必要があります。

In [ ]:
! pip install azure_ai_formrecognizer

## レシートを分析する

これで、Form Recognizerを使ってレシートを分析する準備ができました。

In [ ]:
from python_code import form_recognizer
import os

try:
    # 画像データを読み込む
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
    with open(image_path, "rb") as f:
        data = f.read()
    
    # 画像データをフォームレコグナイザーに送信する
    receipt_data = form_recognizer.get_form_data(form_key, form_endpoint, data)

    # 結果を印刷する
    print('\n')
    for field in receipt_data:
        print(field, receipt_data[field])

except Exception as ex:
    print('Error:', ex)

From Recognizerは、フォーム内のデータを解釈し、加盟店の住所、電話番号、取引日時、ラインアイテム、小計、税金、合計金額を正しく識別することができます。

> **Note**: Form Recognizerにリクエストを送信し、結果を処理するためのコードに興味がある場合は、**python_code**フォルダ内の**form_recognizer.py**ファイルを参照してください。

## More Information

Form Recognizerサービスの詳細については、[Form Recognizerのドキュメント](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/index)を参照してください。